In [24]:
import random
import twitter
import emoji
import gensim
import unicodedata
import html
from keras.preprocessing.text import text_to_word_sequence
import re

In [2]:
# Fill these in!

CONSUMER_KEY = 'xbMuxcJpRTiVGt2C2EYnA'
CONSUMER_SECRET = '2DbQTsvIptkPTdaUcos8DDvQH9fzO0hNjJpUT2uVzQ'
ACCESS_TOKEN = '7319442-EDm4CPxL7W4KkZcGWRMJNVHp88W5OH9vgblu898fg'
ACCESS_SECRET = '5ZxJSbqXhG7uhgXzTFWf9XhkfsxxinlPRXyDTzbA9w'

In [13]:
auth=twitter.OAuth(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    token=ACCESS_TOKEN,
    token_secret=ACCESS_SECRET,
)

status_stream = twitter.TwitterStream(auth=auth).statuses
next(status_stream.sample()).keys()

dict_keys(['source', 'in_reply_to_status_id', 'favorite_count', 'place', 'created_at', 'id', 'favorited', 'retweet_count', 'in_reply_to_user_id_str', 'display_text_range', 'in_reply_to_screen_name', 'contributors', 'in_reply_to_user_id', 'truncated', 'lang', 'coordinates', 'retweeted', 'timestamp_ms', 'id_str', 'text', 'is_quote_status', 'user', 'in_reply_to_status_id_str', 'filter_level', 'entities', 'geo'])

In [41]:
RE_URL = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
RE_WHITESPACE = re.compile(r'\s+')

def strip_accents(s):
     return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

class TokensYielder(object):
    def __init__(self, tweet_count, stream):
        self.tweet_count = tweet_count
        self.stream = stream

    def __iter__(self):
        print('!')
        count = self.tweet_count
        for tweet in self.stream:
            if tweet.get('lang') != 'en':
                continue
            text = tweet['text']
            text = html.unescape(text)
            text = RE_WHITESPACE.sub(' ', text)
            text = RE_URL.sub(' ', text)
            text = strip_accents(text)
            text = ''.join(ch for ch in text if ord(ch) < 128)
            if text.startswith('RT '):
                text = text[3:]
            text = text.strip()
            if text:
                yield text_to_word_sequence(text)
                count -= 1
                if count <= 0:
                    break

for t in TokensYielder(3, twitter.TwitterStream(auth=auth).statuses.sample()):
    print(t)

!
['ico', 'is', 'open', 'to', 'investors']
['louisegrimmer', 'the', 'grocery', 'dr', 'thehenryjones', 'utas', 'tasmanianretail', 'so', 'excited']
['itscaasho', 'say', 'wallahi', "he's", 'talking', 'to', 'a', 'dead', 'dog', 'loooool']


In [58]:
tweets += list(TokensYielder(70000, twitter.TwitterStream(auth=auth).statuses.sample()))

!


In [59]:
model = gensim.models.Word2Vec(tweets, 
                               workers=5,
                               min_count=2,
                              )
model.save('twitter_stream_w2v.model')

In [72]:
model.wv.most_similar(positive=['love'], topn=5)

[('hate', 0.7243724465370178),
 ('loved', 0.7227891087532043),
 ('453', 0.707709789276123),
 ('melanin', 0.7069753408432007),
 ('appreciate', 0.696381688117981)]